In [22]:
# Import the necessary libraries
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
 
 
# Define a function to wait for the page to fully load
def wait_for_page_load():
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'card')))
 
def extract_data(cards):
    # Initialize lists to store data
    bid_numbers = []
    ra_numbers = []
    items = []
    quantities = []
    departments = []
    start_dates = []
    end_dates = []
 
    # Extract data from each card
    for card in cards:
 
        bid_number = card.find('a', class_='bid_no_hover').text.strip()
        bid_numbers.append(bid_number)
       
 
        try:
            ra_number = card.find_all('a', class_='bid_no_hover')[1].text.strip()
            ra_numbers.append(ra_number)
        except:
            ra_numbers.append(0)
 
       
        temp = list(i['data-content'] for i in card.findAll("a", attrs={"data-content":True}))
        if temp!=[]:
            item = temp[0]
        else:
            item = card.findAll('div', class_="row")[1].get_text(strip=True).split(":")[1]
        items.append(item)
       
        quantity = card.find('strong', string='Quantity:').next_sibling.strip()
        quantities.append(quantity)
       
        department = card.find('strong', string='Department Name And Address:').find_next('div').text.strip()
        departments.append(department)
       
        start_date = card.find('strong', string='Start Date:').find_next_sibling().text.strip()
        start_dates.append(start_date)
       
        end_date = card.find('strong', string='End Date:').find_next_sibling().text.strip()
        end_dates.append(end_date)
 
    # Create a DataFrame
    data = {
        'Bid Number': bid_numbers,
        'RA Number': ra_numbers,
        'Items': items,
        'Quantity': quantities,
        'Department': departments,
        'Start Date': start_dates,
        'End Date': end_dates
    }
    df = pd.DataFrame(data)
 
    return df
 
 
 
browser = webdriver.Chrome()
browser.get("https://bidplus.gem.gov.in/all-bids")
 
# Wait for the page to fully load
wait_for_page_load()
 
# Creating empty DataFrame with columns
columns1 = ['Bid Number', 'RA Number', 'Items', 'Quantity', 'Department', 'Start Date', 'End Date']
df1 = pd.DataFrame(columns=columns1)
 
 
soup = BeautifulSoup(browser.page_source, 'html.parser')
total_no_of_pages = int(soup.find_all("a", class_="page-link")[-2].decode_contents())
cards = None
 
# for loop to iterage through the pages
for i in range(2,10):
# for i in range(2, total_no_of_pages):
   
    wait_for_page_load()    
 
    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # Find all card elements
    cards = soup.find_all('div', class_='card')
 
    data = extract_data(cards)
 
    df1 = pd.concat([df1, data], ignore_index=True)
 
    # Find the button and click
    button = browser.find_element(By.LINK_TEXT, str(i))
    button.click()
 
 
 
# Close the browser after the loop finishes
browser.quit()

# Save the data as a CSV file
df1.to_csv('data.csv',index=False)

In [18]:
df1['Items'].unique()

array(['Television (TV) (V2)',
       'Viton O Ring-26200970,Viton O Ring-51004200,Viton O Ring-53307537,Viton O Ring-35003700,Viton O Rin',
       'Field Search Light (MHA)',
       '921043610000 BW LR 90DEG ELL OD219.1X8.18 SA234WPB,921044290200 BW LR 90DEG ELL OD88.9X5.49 SA234WP',
       'Supply of bogie assemblies for complete VANDE Bharat trainset',
       'Online UPS (V2)',
       'Custom Bid for Services - REPAIR OF OFC CABLE AS PER SCOPE OF WORK AND BOQ',
       'Fd Security Light,Cement 43 Grade OPC,Coarse Sand free from vegetation,Coarse aggregate 20mm graded',
       'Super Structure as per store list and specification att,B and R items as per store list att,E and M',
       'Grossing Instruments,Laboratory manual cell counter,Centrifuge 15x16,Antigen Retrieval Pressure Coo',
       'Short Term Cab & Taxi Hiring Services - Sedan; Local; 80KMS X 08 HRS,Short Term Cab & Taxi Hiring S',
       'CCTV camera systems for various sub registrar offices across Gujarat state on behal

In [38]:
df1['Items'].unique()

0                              Field Search Light (MHA)
1     Supply of bogie assemblies for complete VANDE ...
2                                       Online UPS (V2)
3     Custom Bid for Services - REPAIR OF OFC CABLE ...
4     Fd Security Light,Cement 43 Grade OPC,Coarse S...
                            ...                        
75                                      32" LED MONITOR
76    PVC Copper Cable Single and Multi Core Circula...
77    Customized AMC/CMC for Pre-owned Products - UP...
78    6 Female 90 Degree,8 Female 90 Degree,10 Male ...
79    Stainless Steel Dining Table With Fixed Chair,...
Name: Items, Length: 80, dtype: object

In [45]:
filtered_df = df1[df1.apply(lambda row: row.astype(str).str.contains('REPAIR ', case=False).any(), axis=1)]
filtered_df['Items'].iloc[0]


'Custom Bid for Services - REPAIR OF OFC CABLE AS PER SCOPE OF WORK AND BOQ'

In [44]:
filtered_df

,Bid Number,RA Number,Items,Quantity,Department,Start Date,End Date
3,GEM/2024/B/4480076,GEM/2024/R/325684,Custom Bid for Services - REPAIR OF OFC CABLE ...,1,Ministry of DefenceDepartment of Defence Produ...,31-03-2024 6:00 PM,02-04-2024 12:44 PM
41,GEM/2024/B/4746538,0,Repair and Maintenance of Plant/ Systems/Equip...,1,PMODepartment of Atomic Energy,12-03-2024 4:14 PM,02-04-2024 1:00 PM


In [47]:
print("hello")

hello


In [ ]:
filtered_df = df1[df1.apply(lambda row: row.astype(str).str.contains('Repair',case=False).any(),axis =1)]
filtered_df['Items'].iloc[0]

In [1]:
# Main code-

In [ ]:
import os
from flask import Flask, render_template,request,jsonify
import pandas as pd
from extract import startfun
 
app = Flask(__name__)
 
items = []
department = []
df1 = None
 
@app.route('/')
def index():
    items = ["Transformer","Services","Generation","REPAIRS","MACHINE"]
    department = ["Ministry of Coal","Heavy Industries","Petroleum and Natural Gas","Civil Aviation","EducationDepartment"]

    return render_template('index.html',items= items, department= department)
 
@app.route('/sendData', methods=['POST'])
def receive_data():
    global df1
    data = request.json
    input1 = data.get('input1')
    input2 = data.get('input2')
    input1 = input1.split('\n')
    input2 = input2.split('\n')
    items,department,df1 = startfun()
    items.extend(input1)
    department.extend(input2)
    columns1 = ['Bid Number', 'RA Number', 'Items', 'Quantity', 'Department', 'Start Date', 'End Date']
    df2 = pd.DataFrame(columns=columns1)
    temp = f"{items}\n{department}"
    temp = temp.split('\n')
    temp = items+department
    for i in temp:
        filtered_df = df1[df1.apply(lambda row: row.astype(str).str.contains(i, case=False).any(), axis=1)]
        df2 = pd.concat([df2, filtered_df], ignore_index=True)
 
    json_df = df2.to_json(orient='records')  
 
    # print(temp)
    # print(df2.shape)
   
 
    # items,department,json_df = finddataframe(items,department,df1)
 
 
    return jsonify({'items': items, 'department': department, 'dataframe': json_df})
 
@app.route('/refreshData', methods=['POST'])
def refresh_data():
    data = request.json
    selected_items = data.get('selectedItems')
    selected_departments = data.get('selectedDepartments')
 
    # Process selected values as needed
    print("Selected Items:", selected_items)
    print("Selected Departments:", selected_departments)
    print(df1.shape)
    columns1 = ['Bid Number', 'RA Number', 'Items', 'Quantity', 'Department', 'Start Date', 'End Date']
    df2 = pd.DataFrame(columns=columns1)
    temp = selected_items+selected_departments
    for i in temp:
        filtered_df = df1[df1.apply(lambda row: row.astype(str).str.contains(i, case=False).any(), axis=1)]
        df2 = pd.concat([df2, filtered_df], ignore_index=True)
 
    json_df = df2.to_json(orient='records') 
    print(df2.shape)
 
    # items,department,json_df = finddataframe(selected_items,selected_departments)
   
    # return jsonify({'items': items, 'department': department, 'dataframe': json_df})
    return jsonify({'dataframe': json_df})
 
 
if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)